# 解析 DrugBank XML数据库并提取至 TSV文件

使用 Python 3 运行以避免在使用 csv 模块写入文件时出现非 ASCII 字符错误。

In [1]:
import os
import csv
import gzip
import collections
import re
import io
import json
import xml.etree.ElementTree as ET

import requests
import pandas

In [6]:
with open('../data/drugbank.xml', 'rb') as xml_file:
    tree = ET.parse(xml_file)
root = tree.getroot()

In [19]:
ns = '{http://www.drugbank.ca}'

rows = list()
for i, drug in enumerate(root):
    row = collections.OrderedDict()
    assert drug.tag == ns + 'drug'
    row['type'] = drug.get('type')
    row['drugbank_id'] = drug.findtext(ns + "drugbank-id[@primary='true']")
    row['name'] = drug.findtext(ns + "name")
    row['description'] = drug.findtext(ns + "description")
    row['indication_text'] = drug.findtext(ns + "indication")
    row['groups'] = [group.text for group in
        drug.findall("{ns}groups/{ns}group".format(ns = ns))]
    row['atc_codes'] = [code.get('code') for code in
        drug.findall("{ns}atc-codes/{ns}atc-code".format(ns = ns))]
    row['categories'] = [x.findtext(ns + 'category') for x in
        drug.findall("{ns}categories/{ns}category".format(ns = ns))]

    rows.append(row)

In [20]:
def collapse_list_values(row):
    for key, value in row.items():
        if isinstance(value, list):
            row[key] = '|'.join(value)
    return row

rows = list(map(collapse_list_values, rows))

In [21]:
rows[0]

OrderedDict([('type', 'biotech'),
             ('drugbank_id', 'DB00001'),
             ('name', 'Lepirudin'),
             ('description',
              'Lepirudin is identical to natural hirudin except for substitution of leucine for isoleucine at the N-terminal end of the molecule and the absence of a sulfate group on the tyrosine at position 63. It is produced via yeast cells. Bayer ceased the production of lepirudin (Refludan) effective May 31, 2012.'),
             ('indication_text',
              'For the treatment of heparin-induced thrombocytopenia'),
             ('groups', 'approved'),
             ('atc_codes', 'B01AE02'),
             ('categories',
              'Amino Acids, Peptides, and Proteins|Anticoagulants|Antithrombin Proteins|Antithrombins|Blood and Blood Forming Organs|Cardiovascular Agents|Enzyme Inhibitors|Fibrin Modulating Agents|Hematologic Agents|Peptides|Protease Inhibitors|Proteins|Serine Protease Inhibitors|Serpins|Thrombin Inhibitors')])

In [22]:
columns = ['drugbank_id', 'name', 'type', 'groups', 'atc_codes', 'categories', 'indication_text', 'description']
drugbank_df = pandas.DataFrame.from_dict(rows)[columns]
drugbank_df.head()

drugbank_id                 name     type                    groups  \
0     DB00001            Lepirudin  biotech                  approved   
1     DB00002            Cetuximab  biotech                  approved   
2     DB00003         Dornase alfa  biotech                  approved   
3     DB00004  Denileukin diftitox  biotech  approved|investigational   
4     DB00005           Etanercept  biotech  approved|investigational   

  atc_codes                                         categories  \
0   B01AE02  Amino Acids, Peptides, and Proteins|Anticoagul...   
1   L01XC06  Amino Acids, Peptides, and Proteins|Antibodies...   
2   R05CB13  Amino Acids, Peptides, and Proteins|Cough and ...   
3   L01XX29  ADP Ribose Transferases|Amino Acids, Peptides,...   
4   L04AB01  Agents reducing cytokine levels|Amino Acids, P...   

                                     indication_text  \
0  For the treatment of heparin-induced thrombocy...   
1  Cetuximab, used in combination with irinotecan...   
2  Used as adjunct therapy in the treatment of cy...   
3         For treatment of cutaneous T-cell lymphoma   
4  Etanercept is indicated for the treatment of m...   

                                         description  
0  Lepirudin is identical to natural hirudin exce...  
1  Cetuximab is an epidermal growth factor recept...  
2  Dornase alfa is a biosynthetic form of human d...  
3  A recombinant DNA-derived cytotoxic protein co...  
4  Dimeric fusion protein consisting of the extra...

In [24]:
# 获取批准的小分子药物
drugbank_slim_df = drugbank_df[
    drugbank_df.groups.map(lambda x: 'approved' in x) &
    drugbank_df.type.map(lambda x: x == 'small molecule')
]
drugbank_slim_df.head()

drugbank_id          name            type                    groups  \
5      DB00006   Bivalirudin  small molecule  approved|investigational   
6      DB00007    Leuprolide  small molecule  approved|investigational   
13     DB00014     Goserelin  small molecule                  approved   
25     DB00027  Gramicidin D  small molecule                  approved   
33     DB00035  Desmopressin  small molecule                  approved   

          atc_codes                                         categories  \
5           B01AE06  Amino Acids, Peptides, and Proteins|Anticoagul...   
6   L02AE51|L02AE02  Adrenal Cortex Hormones|Agents Causing Muscle ...   
13          L02AE03  Adrenal Cortex Hormones|Amino Acids, Peptides,...   
25          R02AB30  Amino Acids, Peptides, and Proteins|Anti-Bacte...   
33          H01BA02  Agents that produce hypertension|Amino Acids, ...   

                                      indication_text  \
5   For treatment of heparin-induced thrombocytope...   
6   Leuprolide is indicated for the palliative tre...   
13  Goserelin is indicated for:\r\n\r\n- Use in co...   
25  For treatment of skin lesions, surface wounds ...   
33  - Indicated for the treatment of nocturia due ...   

                                          description  
5   Bivalirudin is a synthetic 20 residue peptide ...  
6   Leuprolide is a synthetic 9-residue peptide an...  
13  Goserelin is a synthetic hormone. In men, it s...  
25  Gramcidin D is a heterogeneous mixture of thre...  
33  Desmopressin (dDAVP), a synthetic analogue of ...

In [25]:
# 保存所有药物信息至drugbank_tsv文件
drugbank_df.to_csv('../data/drugbank.tsv', sep='\t', index=False)

# 保存批准小分子药物信息至drug-slim.tsv文件
drugbank_slim_df.to_csv('../data/drugbank-slim.tsv', sep='\t', index=False)

## Extract protein information

In [27]:
protein_rows = list()
for i, drug in enumerate(root):
    drugbank_id = drug.findtext(ns + "drugbank-id[@primary='true']")
    for category in ['target', 'enzyme', 'carrier', 'transporter']:
        proteins = drug.findall('{ns}{cat}s/{ns}{cat}'.format(ns=ns, cat=category))
        for protein in proteins:
            row = {'drugbank_id': drugbank_id, 'category': category}
            row['organism'] = protein.findtext('{}organism'.format(ns))
            row['known_action'] = protein.findtext('{}known-action'.format(ns))
            actions = protein.findall('{ns}actions/{ns}action'.format(ns=ns))
            row['actions'] = '|'.join(action.text for action in actions)
            uniprot_ids = [polypep.text for polypep in protein.findall(
                "{ns}polypeptide/{ns}external-identifiers/{ns}external-identifier[{ns}resource='UniProtKB']/{ns}identifier".format(ns=ns))]            
            if len(uniprot_ids) != 1:
                continue
            row['uniprot_id'] = uniprot_ids[0]
            protein_rows.append(row)

protein_df = pandas.DataFrame.from_dict(protein_rows)

In [28]:
protein_df

drugbank_id     category                              organism  \
0         DB00001       target                                Humans   
1         DB00002       target                                Humans   
2         DB00002       target                                Humans   
3         DB00002       target                                Humans   
4         DB00002       target                                Humans   
...           ...          ...                                   ...   
27164     DB15982  transporter                                Humans   
27165     DB16019       target                                Humans   
27166     DB16353       target                                Humans   
27167     DB16370       target                                Humans   
27168     DB16385       target  Zaire ebolavirus (strain Mayinga-76)   

      known_action     actions uniprot_id  
0              yes   inhibitor     P00734  
1              yes  antagonist     P00533  
2          unknown                 O75015  
3          unknown                 P02745  
4          unknown                 P02746  
...            ...         ...        ...  
27164      unknown   substrate     Q9UNQ0  
27165          yes      binder     P07288  
27166          yes   inhibitor     P23467  
27167          yes   inhibitor     P28907  
27168          yes  antagonist     Q05320  

[27169 rows x 6 columns]

In [31]:
path = '../data/proteins.tsv'
protein_df.to_csv(path, sep='\t', index=False)

In [34]:
# 蛋白质种类数
len(set(protein_df.uniprot_id))

5004

In [32]:
# 具有protient特征的药物数
len(set(protein_df.drugbank_id))

7919